# Assignment 2
In this task a cancer cell detection system will be created.

Luke Mason - s3630120 \
Shekhar Hazari - s3831855 asdasdasdasd

#### Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler

#### Initialise helper functions¶

In [ ]:
def holdout_validation(train_size, X, Y):
    
    # Get the difference between train and test size, together both make 100% of the raw data set.
    test_size = 1 - train_size
    
    # Split into train and test for normal and target fields set, left overs are for validation and test.
    with pd.option_context('mode.chained_assignment', None):
        train_X, X_test_, train_Y, Y_test_ = train_test_split(X, Y, test_size=test_size, shuffle=True,random_state=0)

    # Split the test data into validation test and test.
    # Do for target field and normal fields.
    with pd.option_context('mode.chained_assignment', None):
         val_X, test_X, val_Y, test_Y = train_test_split(X_test_, Y_test_, test_size=0.5, shuffle=True, random_state=29)
    return train_X, val_X, test_X, train_Y, val_Y, test_Y

def hist_plot(title, columns, colours, nrows, ncols, index_pos, left=0.125, top=0.9, bottom=0.1, right=0.9, wspace=0.3, hspace=0.7):
    """
    Plots a histogram.
    :param title:     The title of the histogram.
    :param columns:   The data columns to plot on the histogram.
    :param colours:   The colours for each data column on the histogram.
    :param nrows:     The number of rows in the presentation table.
    :param ncols:     The number of columns in the presentation table.
    :param index_pos: The index of the histogram on the
    :param left:      How far to move away from left.
    :param top:       How far to move away from top.
    :param bottom:    How far to extend towards bottom.
    :param right:     How far to extend boundary towards right.
    :param wspace:    Height ratio spacing between plots.
    :param hspace:    Width ratio spacing between plots.
    :return:
    """
    plt.subplot(nrows,ncols,index_pos)
    plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)
    for index_pos, column in enumerate(columns):
        plt.hist(column, alpha=0.3, color=colours[index_pos % len(colours)], density=True)
    plt.title(title)
    plt.xticks(rotation='vertical')


#### Unzip zipped data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change according to your directory structure
!ls /content/drive/'My Drive'/Collab Notebooks/ML-A2/

In [ ]:
# Change according to your directory structure
!cp /content/drive/'My Drive'/Collab Notebooks/ML-A2/Image_classification_data.zip .
!unzip -q -o Image_classification_data.zip
!rm Image_classification_data.zip
!ls

In [ ]:
# Uncomment if on local machine or AWS
import zipfile
import os

if (not os.path.exists("./data_labels_maindata.csv")):
    with zipfile.ZipFile('./Image_classification_data.zip', 'r') as zip_ref:
        zip_ref.extractall('./')

#### Load the main dataset
This data contains both the cell type and isCancerous labels.

In [ ]:
raw_dataframe_main = pd.read_csv('./data_labels_maindata.csv', delimiter=',')
raw_dataframe_main.describe()

,InstanceID,patientID,cellType,isCancerous
count,9896.000000,9896.000000,9896.000000,9896.000000
mean,10193.880154,29.762025,1.501516,0.412187
std,6652.912660,17.486553,0.954867,0.492253
min,1.000000,1.000000,0.000000,0.000000
25%,4135.750000,14.000000,1.000000,0.000000
50%,9279.500000,26.000000,2.000000,0.000000
75%,16821.250000,47.000000,2.000000,1.000000
max,22444.000000,60.000000,3.000000,1.000000


In [ ]:
raw_dataframe_main.head()

,InstanceID,patientID,ImageName,cellTypeName,cellType,isCancerous
0,22405,1,22405.png,fibroblast,0,0
1,22406,1,22406.png,fibroblast,0,0
2,22407,1,22407.png,fibroblast,0,0
3,22408,1,22408.png,fibroblast,0,0
4,22409,1,22409.png,fibroblast,0,0


#### Load the extra dataset
This dataset does not contain the cellType labels.

In [ ]:
raw_dataframe_extra = pd.read_csv('./data_labels_extraData.csv', delimiter=',')
raw_dataframe_extra.describe()

,InstanceID,patientID,isCancerous
count,10384.000000,10384.00000,10384.000000
mean,12087.866333,80.38203,0.287943
std,6173.866838,9.40388,0.452826
min,1631.000000,61.00000,0.000000
25%,6655.750000,71.00000,0.000000
50%,12377.500000,81.00000,0.000000
75%,16374.250000,88.00000,1.000000
max,22235.000000,99.00000,1.000000


In [ ]:
raw_dataframe_extra.head()

,InstanceID,patientID,ImageName,isCancerous
0,12681,61,12681.png,0
1,12682,61,12682.png,0
2,12683,61,12683.png,0
3,12684,61,12684.png,0
4,12685,61,12685.png,0


### Data Setup
ImageName, InstanceID, features are excluded because they won't help in EDA since they are all unique values and are not information that is useful by themselves.

The raw data frame will be used to get the image names to map them to the images for the feature gathering.

In [ ]:
# Construct non-target and target datasets. Y is target labels, X is non-target.
df_main_features = raw_dataframe_main.drop(['InstanceID', 'isCancerous'], axis=1)
df_main_target_label = raw_dataframe_main['isCancerous']

In [ ]:
# Construct non-target and target datasets. Y is target labels, X is non-target.
df_extra_features = raw_dataframe_extra.drop(['InstanceID', 'isCancerous'], axis=1)
df_extra_target_label = raw_dataframe_extra['isCancerous']

### 1 - Exploratory Data Analysis
The data is assumed to be made of entirely by newborn babies according to the assignment forums.\

#### 1.1 - Exploring the distribution of the data
In this section the data's distribution of values are examined. Observations and folowwing conclusions are made to be used for justifying the decisions made in the rest of the report.

##### 1.1.1 Exploring unique values and data types
Here the fields display the unique values there are and their frequency.

In [ ]:
for field in df_main_features.columns:
    df = df_main_features.groupby(field).nunique()
    print(df)

           ImageName  cellTypeName  cellType
patientID                                   
1                 19             3         3
2                 33             3         3
3                136             3         3
4                127             3         3
5                169             4         4
6                198             4         4
7                253             4         4
8                332             4         4
9                348             4         4
10               302             4         4
11                56             3         3
12               130             3         3
13               180             4         4
14               207             4         4
15               125             3         3
16               111             3         3
17               310             4         4
18               320             4         4
19               158             3         3
20               325             3         3
21        

In [ ]:
hv_train_X, hv_val_X, hv_test_X, hv_train_Y, hv_val_Y, hv_test_Y = holdout_validation(0.8, df_main_features, df_main_target_label)

# Graph the results
plt.figure(figsize=(20,40))
for i, col in enumerate(df_main_features.columns):
    hist_plot(col, [hv_train_X[col], hv_val_X[col], hv_test_X[col]], ['r', 'g', 'b'], 8, 5, i+1)

NameError: name 'hist_plot' is not defined

<Figure size 1440x2880 with 0 Axes>

In [ ]:
print("Train data : {}, Val Data: {}, Test Data: {}".format(hv_train_X.shape[0], hv_val_X.shape[0], hv_test_X.shape[0]))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

# TODO haven't figured out how to get the image path yet
train_generator = train_datagen.flow_from_dataframe(
        dataframe=hv_train_X,
        directory='./patch_images/',
        x_col="ImageName",
        y_col="Class",
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_df_main.flow_from_dataframe(
        dataframe=hv_val_X,
        directory='./',
        x_col="ImageName",
        y_col="Class",
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')

ModuleNotFoundError: No module named 'tensorflow'